This notebook documents the extraction and formatting of 250 emails from the Apache Camel archives into a dataframe.

The original 250 emails need to be downloaded in MBOX format. Their ground truth labels must be downloaded in CSV.

The emails are available at http://mail-archives.apache.org/mod_mbox/camel-dev/201704.mbox/browser, over the period 2017-04-14 10:42:39 UTC to 2017-04-19 13:27:37 UTC. The labels are available at https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6366754/ with extraction instructions.

In [ ]:
pathToEmails  = ".data/mailbox/" # Original MBOX folder path
pathToCleanup = "data/camel_emails.csv" # Output CSV path for extracted emails from MBOX
keywords_labels_path = "labels/keyword_labels.csv" # Input path to original keywords labels CSV
consensus_metalabels_path = "labels/consensus_metalabels.csv" # Input path to original labels CSV
consensus_traces_path = "labels/consensus_traces.csv" # Input path to original instances labels CSV
consensus_workflows_path = "labels/consensus_workflows.csv" # Input path to original workflows labels CSV
workflows_construction_path = "labels/workflows_construction.csv" # Input path to original workflows construction CSV

save_path_emails_df = "data/camel_emails.csv" # Output CSV path for extracted emails from MBOX
save_path_final_df_csv = "data/camel_emails_final_na_clean.csv" # Output CSV path for final extracted emails and labels

# Imports

In [ ]:
from tidyextractors.tidymbox import MboxExtractor
import pandas as pd
import datetime
import pytz
import csv
import os
import mailbox
import re
import pandas as pd

# Load emails

In [ ]:
keywords_labels = pd.read_csv(keywords_labels_path)
consensus_metalabels = pd.read_csv(consensus_metalabels_path)
consensus_traces = pd.read_csv(consensus_traces_path)
consensus_workflows = pd.read_csv(consensus_workflows_path)
workflows_construction = pd.read_csv(workflows_construction_path)

In [ ]:
# Email extraction functions from https://github.com/phildeutsch/mbox-analysis

def clean_addresses(addresses, lookupcsv):
    if addresses is None:
        return []
    addresses = addresses.replace("\'", "")
    addressList = re.split('[,;]', addresses)
    cleanList = []
    for address in addressList:
        cleanAddress = clean_address(address, lookupcsv)
        cleanList.append(cleanAddress)
    return cleanList

def clean_address(address, lookupcsv):
    address = address.replace("<", "")
    address = address.replace(">", "")
    address = address.replace("\"", "")
    address = address.replace("\n", " ")
    address = address.replace("MAILER-DAEMON", "")
    address = address.lower().strip()

    with open(lookupcsv, 'rt') as lookupfile:
        lookupdata = lookupfile.readlines()
    for line in lookupdata:
        name = line.split(',')[0]
        if address == name:
            address = line.split(',')[-1].strip()

    email = None
    for word in address.split(' '):
        emailRegex = re.compile(
            "^[a-zA-Z0-9._%-]+@[a-zA-Z0-9._%-]+.[a-zA-Z]{2,6}$"
            )
        email = re.match(emailRegex, word)
        if email is not None:
            cleanEmail = email.group(0)
    if email is None:
        if address.split(' ')[-1].find('@') > -1:
            cleanEmail = address.split(' ')[-1].strip()
        elif address.split(' ')[-1].find('?') > -1:
            cleanEmail = 'n/a'
        else:
            cleanEmail = address
              
    return cleanEmail

def get_body(message):
    try:
        sm = str(message)
        body = sm
        body = body.replace("=20\n", "")
        body = body.replace("=FC", "ü")
        body = body.replace("=F6", "ö")
        body = body.replace("=84", "\"")
        body = body.replace("=94", "\"")
        body = body.replace("=96", "-")
        body = body.replace("=92", "\'")
        body = body.replace("=93", "\"")
        body = body.replace("=E4", "ä")
        body = body.replace("=DF", "ss")
        body = body.replace("=", "")
        body = body.replace("\"", "")
        body = body.replace("\'", "")
    except:
        body = "N/A"
    
    return body

def write_table(mboxfile, mailTable, pathToCleanup):
    for message in mailbox.mbox(mboxfile):
        cleanFrom = clean_address(message['From'], pathToCleanup)
        cleanTo = clean_addresses(message['To'], pathToCleanup)
        cleanCc = clean_addresses(message['Cc'], pathToCleanup)
        mailTable.append([
            cleanFrom,
            cleanTo,
            cleanCc,
            message['Date'],
            message['Subject'],
            get_body(message)
            ])

Extract MBOX files into dataframe.

In [ ]:
mboxfiles = [os.path.join(dirpath, f) for dirpath, dirnames, files in os.walk(pathToEmails) for f in files if f.endswith('mbox')]
mailTable = []

for mboxfile in mboxfiles:
    write_table(mboxfile, mailTable, pathToCleanup)

email_df = pd.DataFrame(mailTable)
email_df.columns = ['From', 'To', 'Cc', 'Date', 'Subject', 'Body']
email_df['NumTo'] = email_df['To'].map(lambda i: len(i))

Clean dataframe.

In [ ]:
email_clean = email_df[['Date', 'From', 'To', 'Subject', 'Body']]

# Isolate various date string format and normalize
email_clean_date_without_tz_single_digits = email_clean[email_clean.Date.str.len() == 30]
email_clean_date_without_tz_single_digits['Date'] = email_clean_date_without_tz_single_digits.Date.str[:5] + '0' + email_clean_date_without_tz_single_digits.Date.str[5:]
email_clean_date_without_tz_double_digits = email_clean[email_clean.Date.str.len() == 31]
email_clean_date_with_tz_single_digits = email_clean[email_clean.Date.str.len() == 36]
email_clean_date_with_tz_single_digits['Date'] = email_clean_date_with_tz_single_digits.Date.str[:5] + '0' + email_clean_date_with_tz_single_digits.Date.str[5:30]
email_clean_date_with_tz_double_digits = email_clean[email_clean.Date.str.len() == 37]
email_clean_date_with_tz_double_digits['Date'] = email_clean_date_with_tz_double_digits.Date.str.replace("  ", ' 0', regex=False)
email_clean_date_with_tz_double_digits['Date'] = email_clean_date_with_tz_double_digits.Date.str[:31]

# Convert to datetime
email_clean_date_without_tz_single_digits['Date'] = pd.to_datetime(email_clean_date_without_tz_single_digits['Date'], format='%a, %d %b %Y %H:%M:%S %z', errors='coerce')
email_clean_date_without_tz_double_digits['Date'] = pd.to_datetime(email_clean_date_without_tz_double_digits['Date'], format='%a, %d %b %Y %H:%M:%S %z', errors='coerce')
email_clean_date_with_tz_single_digits['Date'] = pd.to_datetime(email_clean_date_with_tz_single_digits['Date'], format='%a, %d %b %Y %H:%M:%S %z', errors='coerce')
email_clean_date_with_tz_double_digits['Date'] = pd.to_datetime(email_clean_date_with_tz_double_digits['Date'], format='%a, %d %b %Y %H:%M:%S %z', errors='coerce')

# Concatenate dfs
email_clean = pd.concat([
    email_clean_date_without_tz_single_digits,
    email_clean_date_without_tz_double_digits,
    email_clean_date_with_tz_single_digits, 
    email_clean_date_with_tz_double_digits
])

# Filter time interval
email_clean = email_clean[
    (email_clean['Date'] >= datetime.datetime(2017, 4, 14, 10, 42, 39, 0, pytz.UTC)) &
    (email_clean['Date'] <= datetime.datetime(2017, 4, 19, 13, 27, 37, 0, pytz.UTC))
]

C:\Users\patafilm\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\patafilm\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\patafilm\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

Compute email ID for future join with labels.

In [ ]:
email_clean = email_clean.sort_values('Date')
email_clean = email_clean.reset_index()
email_clean.insert(0, 'Email_ID', range(1, len(email_clean)+1))

# Re arrange Email_ID for emails with the same timestamp
email_clean.at[33-1, 'Email_ID'] = 34
email_clean.at[34-1, 'Email_ID'] = 33

email_clean.at[78-1, 'Email_ID'] = 79
email_clean.at[79-1, 'Email_ID'] = 78

email_clean.at[84-1, 'Email_ID'] = 85
email_clean.at[85-1, 'Email_ID'] = 84

email_clean.at[164-1, 'Email_ID'] = 165
email_clean.at[165-1, 'Email_ID'] = 164

email_clean.at[168-1, 'Email_ID'] = 170
email_clean.at[170-1, 'Email_ID'] = 168


Save the clean dataframe.

In [ ]:
email_clean.to_csv(save_path_emails_df, index=False, quoting=csv.QUOTE_ALL)

# Format emails

## Format email body

In [ ]:
def body_extract(x):
    if x['len_body'] > 1:
        return x['Body_clean'][1] 
    return x['Body_clean'][0]

Remove subject line from body.

In [ ]:
test = email_clean.copy()
test['Body_original'] = test['Body'] 
test['Body_clean'] = test['Body'] 

Remove line breaks and body meta data.

In [ ]:
# "Commits" emails body cleaning.
mask_commits = test['To'].apply(lambda x: 'commits@camel.apache.org' in x)
test_commits = test[mask_commits]
test_commits['Body_clean'] = test_commits['Body_clean'].map(lambda x : x[x.find('\nSubject:', 0):] if x.find('\nSubject:', 0) >=0 else x )

# "Dev" emails body cleaning
mask_dev = test['To'].apply(lambda x: 'dev@camel.apache.org' in x)
test_dev = test[mask_dev]
test_dev['Body_clean'] = test_dev['Body_clean'].map(lambda x : x[x.find('\nSubject:', 0):] if x.find('\nSubject:', 0) >=0 else x )

# "Issues" emails body cleaning
mask_issues = test['To'].apply(lambda x: 'issues@camel.apache.org' in x)
test_issues = test[mask_issues]
test_issues['Body_clean'] = test_issues['Body_clean'].map(lambda x : x[x.find('\nSubject:', 0):] if x.find('\nSubject:', 0) >=0 else x )

# "Users" emails body cleaning
mask_users = test['To'].apply(lambda x: 'users@camel.apache.org' in x)
test_users = test[mask_users]
test_users['Body_clean'] = test_users['Body_clean'].map(lambda x : x[x.find('\nSubject:', 0):] if x.find('\nSubject:', 0) >=0 else x )

# Join cleaned emails dfs
test_df = pd.concat([
    test_commits,
    test_dev,
    test_issues,
    test_users
])
test_df = test_df.replace('\\n',' ', regex=True) 
test_df = test_df.replace('\n>',' ', regex=True) 
test_df = test_df.replace('\n',' ', regex=True) 

C:\Users\patafilm\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\patafilm\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\patafilm\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

## Add email labels

Join the downloaded labels, based on email ID.

In [ ]:
email_final = test_df
email_final = email_final.merge(
    keywords_labels, on='Email_ID', how='left'
).merge(
    consensus_metalabels, on='Email_ID', how='left'
).merge(
    workflows_construction.rename(columns={'Date': 'Date_authors'}), on='Email_ID', how='left'
).merge(
    consensus_traces, on='Trace_ID', how='left'
).merge(
    consensus_workflows.rename(columns={'Trace': 'Trace_ID'}), on='Trace_ID', how='left'
)

## Handle NaN

In [ ]:
# Replace missing email body by subject line
df = email_final.copy()
na_body = df[df.Body.isna()]
df_without_na_body = df[~df.Body.isna()]
cols = ['Subject']
na_body['Body_clean'] = na_body[cols].apply(lambda row: '. '.join(row.values.astype(str)), axis=1)
final_df = pd.concat([na_body, df_without_na_body])

# Clean email body on two specific unstructured instances
body_clean_82 = "Hi, Caused by: java.io.FileNotFoundException: E:\XXX\XXXX\XX\XXXX-000001-XXX.zip (Access is denied) This looks as if the process running your camel route does not have access to the directory containing the file (it would need execution rights on the directory). Best regards Stephan -----Original Message----- From: chandangowda [mailto:cgowda299@gmail.com] Sent: Montag, 17. April 2017 12:09 To: users@camel.apache.org Subject: Camel not able to delete files Outside .camel From one route I’m reading ZIP files after successful completion of reading of files,camel not able to delete those files i.e. after moving files inside .camel from outside .camel and throughing following exception.This issue happened only for  first 21 records  out of 1000 records.Please replay ASAP. Stack trace org.apache.camel.component.file.GenericFileOperationFailedException: Cannot store file: E:\XX\XX\XXXX\XXXXX-000001-XXXX.zip at org.apache.camel.component.file.FileOperations.storeFile(FileOperations.java:269) at org.apache.camel.component.file.GenericFileProducer.writeFile(GenericFileProducer.java:278) at org.apache.camel.component.file.GenericFileProducer.processExchange(GenericFileProducer.java:166) at org.apache.camel.component.file.GenericFileProducer.process(GenericFileProducer.java:80) at org.apache.camel.util.AsyncProcessorConverterHelper$ProcessorToAsyncProcessorBridge.process(AsyncProcessorConverterHelper.java:61) at org.apache.camel.processor.SendProcessor.process(SendProcessor.java:110) at org.apache.camel.management.InstrumentationProcessor.process(InstrumentationProcessor.java:72) at org.apache.camel.processor.RedeliveryErrorHandler.process(RedeliveryErrorHandler.java:398) at org.apache.camel.processor.CamelInternalProcessor.process(CamelInternalProcessor.java:191) at org.apache.camel.processor.CamelInternalProcessor.process(CamelInternalProcessor.java:191) at org.apache.camel.component.file.GenericFileConsumer.processExchange(GenericFileConsumer.java:401) at org.apache.camel.component.file.GenericFileConsumer.processBatch(GenericFileConsumer.java:201) at org.apache.camel.component.file.GenericFileConsumer.poll(GenericFileConsumer.java:165) at org.apache.camel.impl.ScheduledPollConsumer.doRun(ScheduledPollConsumer.java:187) at org.apache.camel.impl.ScheduledPollConsumer.run(ScheduledPollConsumer.java:114) at java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:471) at java.util.concurrent.FutureTask.runAndReset(FutureTask.java:304) at java.util.concurrent.ScheduledThreadPoolExecutor$ScheduledFutureTask.access$301(ScheduledThreadPoolExecutor.java:178) at java.util.concurrent.ScheduledThreadPoolExecutor$ScheduledFutureTask.run(ScheduledThreadPoolExecutor.java:293) at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1145) at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:615) at java.lang.Thread.run(Thread.java:745) Caused by: java.io.FileNotFoundException: E:\XXX\XXXX\XX\XXXX-000001-XXX.zip (Access is denied) at java.io.FileOutputStream.open(Native Method) at java.io.FileOutputStream.<init>(FileOutputStream.java:221) at java.io.FileOutputStream.<init>(FileOutputStream.java:171) at org.apache.camel.component.file.FileOperations.prepareOutputFileChannel(FileOperations.java:430) at org.apache.camel.component.file.FileOperations.writeFileByFile(FileOperations.java:353) at org.apache.camel.component.file.FileOperations.storeFile(FileOperations.java:242) ... 21 more -- View this message in context: http://camel.465427.n5.nabble.com/Camel-not-able-to-delete-files-Outside-camel-tp5797768.html Sent from the Camel - Users mailing list archive at Nabble.com."
final_df.at[final_df[final_df['Email_ID'] == 82].index[0], 'Body_clean'] = body_clean_82
na_body_clean = final_df[final_df.Body_clean.isna()]
df_without_na_body_clean = final_df[~final_df.Body_clean.isna()]
na_body_clean['Body_clean'] = na_body_clean['Body'].str[3100:]
final_df_no_na = pd.concat([na_body_clean, df_without_na_body_clean])

# Save and sanity checks

In [ ]:
final_df_no_na.to_csv(save_path_final_df_csv, index=False, quoting=csv.QUOTE_ALL)
df = pd.read_csv(save_path_final_df_csv, quoting=csv.QUOTE_ALL)

print(df.shape)

(250, 62)


In [ ]:
df[['Email_ID', 'Date', 'From', 'To', 'Subject', 'Body_clean', 'Trace_ID', 'Action']].sort_values('Date')

,Email_ID,Date,From,To,Subject,Body_clean,Trace_ID,Action
211,43,2017-04-14 10:15:23-07:00,revathykuberan@gmail.com,['users@camel.apache.org'],Re: Multiple from end points traversing to dif...,Subject: Re: Multiple from end points travers...,9,ask a question
0,1,2017-04-14 10:42:39+00:00,lburgazzoli@apache.org,['commits@camel.apache.org'],camel git commit: HeaderSelectorProducer to su...,Subject: camel git commit: HeaderSelectorProd...,1,commit changes
1,2,2017-04-14 10:52:55+00:00,davsclaus@apache.org,['commits@camel.apache.org'],[1/6] camel git commit: Rename catalog to runt...,Subject: [1/6] camel git commit: Rename catal...,2,commit changes
2,3,2017-04-14 10:52:56+00:00,davsclaus@apache.org,['commits@camel.apache.org'],[2/6] camel git commit: Rename catalog to runt...,Subject: [2/6] camel git commit: Rename catal...,2,commit changes
3,4,2017-04-14 10:52:57+00:00,davsclaus@apache.org,['commits@camel.apache.org'],[3/6] camel git commit: Rename catalog to runt...,Subject: [3/6] camel git commit: Rename catal...,2,commit changes
...,...,...,...,...,...,...,...,...
118,239,2017-04-19 14:21:57+02:00,lburgazzoli@gmail.com,['dev@camel.apache.org'],Re: Camel 2.19 Roadmap,Subject: Re: Camel 2.19 Roadmap To: dev@camel...,24,version release planning
244,242,2017-04-19 14:29:00+02:00,claus.ibsen@gmail.com,['users@camel.apache.org'],Re: Spring Boot > 1.5 fails when camel-swagger...,Subject: Re: Spring Boot > 1.5 fails when cam...,64,ask a question
247,247,2017-04-19 14:35:40+02:00,zoran@regvart.com,['users@camel.apache.org'],Re: Spring Boot > 1.5 fails when camel-swagger...,Subject: Re: Spring Boot > 1.5 fails when cam...,64,provide support
249,249,2017-04-19 15:27:36+02:00,claus.ibsen@gmail.com,['users@camel.apache.org'],Re: Spring Boot > 1.5 fails when camel-swagger...,Subject: Re: Spring Boot > 1.5 fails when cam...,64,ask a question
